# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open('results/' + nn + '_layer_outputs.dat', 'rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()


# Load recording and test data

In [ ]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

# downsample
factor = 1
if factor<1:
    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


# Run Neural Networks and average outputs

In [ ]:
from copy import copy, deepcopy
from RE_PartialRecData2 import RE_PartialRecData2
from sklearn import preprocessing, linear_model
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import EarlyStopping
from fancyimpute import SoftImpute as FI
import time

# how many recordings?
nRecordings = 10
# which layers?
layers = [0,2]
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]

for iLayer in layers:
    # how many neurons from the firs hidden layer?
    subnetSize = [2**x for x in range(np.int(np.log2(layer_outs[iLayer].shape[1])+1))]
    nSubnetSize = len(subnetSize)
    # how many samples per recording?
    nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
    
    rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])
    print('Layer #', iLayer)
    
    for ss in range(nSubnetSize):
        print(subnetSize[ss]) 
        
        for it in range(nIterations):
            start = time.time()
            # copy data - is this necessary?
            layer_outputs = deepcopy(layer_outs)          

            # subsample
            X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], 
                                                              subnetSize[ss], nRecordings, nSamples[ss])
            # add noise
            noise = 0.20
            sigmax = np.std(layer_outputs[iLayer],axis=0)*noise
            X_noise = [np.random.normal(0,sigmax[i],[X_subsample.shape[0]]) for i in range(X_subsample.shape[1])]
            X_noise = np.transpose(np.asarray(X_noise))
            X_corrupted = X_subsample + X_noise
            sigmay = np.std(layer_outputs[oLayer],axis=0)*noise
            Y_noise = [np.random.normal(0,sigmay[i],[Y_subsample.shape[0]]) for i in range(Y_subsample.shape[1])]
            Y_noise = np.transpose(np.asarray(Y_noise))
            Y_corrupted = Y_subsample + Y_noise
    
            # impute X_subsample with soft impute
            (keep_cols,) = np.where(~np.all(np.isnan(X_corrupted), axis=0))
            if ss==nSubnetSize-1:
                X_new = X_corrupted[:,keep_cols]
            else:
                X_new = FI(convergence_threshold=0.01, max_iters=30).complete(X_corrupted[:, keep_cols])

            X_test_new = layer_outs_test[iLayer][:, keep_cols]
            
            # fit ΝΝ
            model = Sequential()
            model.add(Dense(16, input_shape=(X_new.shape[1],), activation='relu'))
            model.add(Dense(16, activation='relu'))
            model.add(Dense(16, activation='relu'))                
            model.add(Dense(10, activation='softmax'))
            
            # Compile model
            model.compile(loss='mean_squared_error', optimizer='adam')
            callbacks = [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0)]
            model.fit(X_new, Y_corrupted, epochs=30, batch_size=16, shuffle=True, verbose=0, 
                      validation_split=0.1, callbacks=callbacks)
            stop = time.time()
            duration = stop-start
            print(duration)
            rmses[it,:,ss] = np.sqrt(np.mean((model.predict(X_test_new) - layer_outs_test[oLayer])**2, axis=0))
        
        print ('predicting, mean RMSEs=%f' %np.mean(rmses[it, :, ss]))
        
        
    fName = 'results/NNMVI_RMSES_Layer' + str(iLayer) + '_nRec' + str(nRecordings) + '_' + nn + \
            '_Noise' + str(int(noise*100)) + '.dat'
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()


('Layer #', 0)
1
[SoftImpute] Max Singular Value of X_init = 1363.015214
[SoftImpute] Iter 1: observed MAE=0.022969 rank=10
[SoftImpute] Iter 2: observed MAE=0.022969 rank=10
[SoftImpute] Iter 3: observed MAE=0.022969 rank=10
[SoftImpute] Iter 4: observed MAE=0.022969 rank=10
[SoftImpute] Iter 5: observed MAE=0.022969 rank=10
[SoftImpute] Iter 6: observed MAE=0.022969 rank=10
[SoftImpute] Iter 7: observed MAE=0.022969 rank=10
[SoftImpute] Iter 8: observed MAE=0.022969 rank=10
[SoftImpute] Iter 9: observed MAE=0.022969 rank=10
[SoftImpute] Iter 10: observed MAE=0.022969 rank=10
[SoftImpute] Iter 11: observed MAE=0.022969 rank=10
[SoftImpute] Iter 12: observed MAE=0.022969 rank=10
[SoftImpute] Iter 13: observed MAE=0.022969 rank=10
[SoftImpute] Iter 14: observed MAE=0.022969 rank=10
[SoftImpute] Iter 15: observed MAE=0.022969 rank=10
[SoftImpute] Iter 16: observed MAE=0.022969 rank=10
[SoftImpute] Iter 17: observed MAE=0.022969 rank=10
[SoftImpute] Iter 18: observed MAE=0.022969 rank=10
